In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import tqdm
from pulp import *


class CreateDemands:
    def __init__(self, low, high, size=(12,3), seed=42):
        self.times = size[0]
        self.n_items = size[1]
        self.low = low
        self.high = high
        self.seed = seed
    
    def generate(self):
        np.random.seed(self.seed)
        
        demands = np.random.randint(self.low, self.high, self.n_items)
        rnd_matrix = np.random.rand(self.n_items, self.times)
        normalized = rnd_matrix / rnd_matrix.sum(axis=1).reshape((-1,1))
        
        self.demands = np.int32(normalized * demands.reshape((-1,1)))
        self.demands = pd.DataFrame(self.demands)
    
        return self

In [2]:
random_demands = CreateDemands(500, 5000, size=(12,3), seed=42).generate()
random_demands.demands

,0,1,2,3,4,5,6,7,8,9,10,11
0,42,42,15,238,165,194,5,266,228,58,49,50
1,264,455,375,253,531,121,253,318,396,682,173,446
2,371,29,380,106,40,594,605,506,190,61,428,275


**Index**

$\begin{split}
& t \quad & t=1,2,...,T\\
& i \quad & i=1,2,...,N
\end{split}$



**Parameters**

$\begin{split}
& d_{it}    \quad & \text{demand for item $i$ at period $t$,}\\
& S^{w}     \quad & \text{major ordering cost at warehouse,}\\
& s^{w}_{i} \quad & \text{minor ordering cost for item $i$ at warehouse,}\\
& s^{r}_{i} \quad & \text{minor ordering cost for item $i$ at retailers,}\\
& h^{w}_{i} \quad & \text{inventory holding cost for item $i$ at warehouse,}\\
& h^{r}_{i} \quad & \text{inventory holding cost for item $i$ at retailers,}\\
& M         \quad & \text{sufficiently large number,}\\
\end{split}$

**Decision Variables**

$\begin{split}
& Q^{w}_{it} \quad & \text{replenishment quantity of item $i$  at period $t$,}\\
& Q^{r}_{it} \quad & \text{delivery quantity of item $i$  at period $t$,}\\
& Y_{t}      \quad & \text{binary variable taking the value 1 if an order is placed for period $t$,}\\
& Z^{w}_{it} \quad & \text{item $i$  at period $t$,}\\
& Z^{r}_{it} \quad & \text{item $i$  at period $t$,}\\
& I^{w}_{it} \quad & \text{item $i$  at period $t$,}\\
& I^{r}_{it} \quad & \text{item $i$  at period $t$,}\\
\end{split}$

In [3]:
class IndividualModel:
    def __init__(self, d, s, sw, sr, hw, hr):
        self.d = d
        self.n_items = d.shape[0]
        self.times = d.shape[1]
        self.qr_star = d.copy()
        
        self.s = s
        self.sw = sw
        self.sr = sr
        
        self.hw = hw
        self.hr = hr
        
        self.m = d.sum().sum()
        self.verbose = False
        
    def retailers(self):
        retailer_cost = 0
        
        for i in range(self.n_items):
            d = self.d.iloc[i, :].copy() # demands for sub problem
            
            # Define problems
            prob = LpProblem('Retailer{}'.format(i+1), LpMinimize)

            qr = LpVariable.dicts('Qw', list(range(self.times)), lowBound=0, cat='Continuous')
            zr = LpVariable.dicts('Zr', list(range(self.times)), cat='Binary')
            ir = LpVariable.dicts('Ir', list(range(self.times)), lowBound=0, cat='Continuous')

            prob += lpSum([zr[t] for t in range(self.times)]) * self.sr[i] \
                    + lpSum([ir[t] for t in range(self.times)]) * self.hr[i]

            prob += 0 + qr[0] - d.iloc[0] == ir[0]

            for t in range(1, self.times):
                prob += ir[t-1] + qr[t] - d.iloc[t] == ir[t]

            for t in range(self.times):
                prob += qr[t] <= zr[t] * self.m

            prob.solve(solver=GUROBI(msg=False, epgap=0.0))
            if self.verbose:
                print('Ratailer{}: {}'.format(i+1, LpStatus[prob.status]))
            
            retailer_cost += value(prob.objective)
            qr_star = [qr[t].varValue for t in range(self.times)]
            self.qr_star.iloc[i, :] = qr_star
            
            del prob
        return retailer_cost
        
    def warehouse(self):
        idx = [(i, t) for i in range(self.n_items) for t in range(self.times)]
        
        prob = LpProblem('Warehouse', LpMinimize)

        qw = LpVariable.dicts('Qw', idx, lowBound=0, cat='Continuous')
        y = LpVariable.dicts('Y', list(range(self.times)), cat='Binary')
        zw = LpVariable.dicts('Zw', idx, cat='Binary')
        iw = LpVariable.dicts('IW', idx, lowBound=0, cat='Continuous')

        prob += lpSum([y[t] for t in range(self.times)]) * self.s \
                      + lpSum([self.sw[i] * zw[i, t] for i in range(self.n_items) for t in range(self.times)]) \
                      + lpSum([self.hw[i] * iw[i, t] for i in range(self.n_items) for t in range(self.times)])

        for i in range(self.n_items):
            prob += 0 + qw[i, 0] - self.qr_star.iloc[i, 0] == iw[i, 0]

        for i in range(self.n_items):
            for t in range(1, self.times):
                prob += iw[i, t-1] + qw[i, t] - self.qr_star.iloc[i, t] == iw[i, t]

        for t in range(self.times):
            prob += lpSum([qw[i, t] for i in range(self.n_items)]) <= y[t] * self.m

        for i in range(self.n_items):
            for t in range(self.times):
                prob += lpSum(qw[i, t]) <= zw[i, t] * self.m

        prob.solve(solver=GUROBI(msg=False, epgap=0.0))
        if self.verbose:
            print('Warehouse: {}'.format(LpStatus[prob.status]))
        
        warehouse_cost = value(prob.objective)
        del prob
        return warehouse_cost

    def solve(self):
        self.retailer_cost = self.retailers()
        self.warehouse_cost = self.warehouse()
        self.total_cost = self.retailer_cost + self.warehouse_cost
        if self.verbose:
            print('Individual model: Optimal')

In [4]:
class IntegratedModel:
    def __init__(self, d, s, sw, sr, hw, hr):
        self.d = d
        self.n_items = d.shape[0]
        self.times = d.shape[1]
        
        self.s = s
        self.sw = sw
        self.sr = sr
        
        self.hw = hw
        self.hr = hr
        
        self.m = d.sum().sum()
        
        self.verbose = False
        
    def create_problem(self):
        idx = [(i, t) for i in range(self.n_items) for t in range(self.times)]
        
        # Define model
        prob = LpProblem('Warehouse', LpMinimize)
        
        # Decision variables
        qw = LpVariable.dicts('Qw', idx, lowBound=0, cat='Continuous') # 240
        qr = LpVariable.dicts('Qr', idx, lowBound=0, cat='Continuous') # 240
        
        y = LpVariable.dicts('Y', list(range(self.times)), cat='Binary') # 12
        zw = LpVariable.dicts('Zw', idx, cat='Binary') # 240
        zr = LpVariable.dicts('Zr', idx, cat='Binary') # 240
        
        iw = LpVariable.dicts('Iw', idx, lowBound=0, cat='Continuous') # 240
        ir = LpVariable.dicts('Ir', idx, lowBound=0, cat='Continuous') # 240

        # Objective function
        prob += lpSum(y[t] for t in range(self.times)) * self.s \
                + lpSum([self.sw[i] * zw[i, t] for i in range(self.n_items) for t in range(self.times)]) \
                + lpSum([self.sr[i] * zr[i, t] for i in range(self.n_items) for t in range(self.times)]) \
                + lpSum([self.hw[i] * iw[i, t] for i in range(self.n_items) for t in range(self.times)]) \
                + lpSum([self.hr[i] * ir[i, t] for i in range(self.n_items) for t in range(self.times)])
        
        # 기초재고식
        for i in range(self.n_items):
            prob += 0 + qw[i, 0] - qr[i, 0] == iw[i, 0]
            prob += 0 + qr[i, 0] - self.d.iloc[i, 0] == ir[i, 0]

        # 기말재고식
        for i in range(self.n_items):
            for t in range(1, self.times):
                prob += iw[i, t-1] + qw[i, t] - qr[i, t] == iw[i, t]
                prob += ir[i, t-1] + qr[i, t] - self.d.iloc[i, t] == ir[i, t]

        for t in range(self.times):
            prob += lpSum(qw[i, t] for i in range(self.n_items)) <= y[t] * self.m

        for i in range(self.n_items):
            for t in range(self.times):
                prob += lpSum(qw[i, t]) <= zw[i, t] * self.m
                prob += lpSum(qr[i, t]) <= zr[i, t] * self.m
        
        return prob
            
    def solve(self):
        prob = self.create_problem()
        prob.solve(solver=GUROBI(msg=False, epgap=0.0))
        if self.verbose:
            print('Integrated model: {}'.format(LpStatus[prob.status]))
        
        self.total_cost = value(prob.objective)
        
        del prob

In [ ]:
d = pd.read_excel('data.xlsx', header=None)

times = d.shape[1]
n_items = d.shape[0]
        
s = 200
sw = [45, 46, 47, 44, 45, 47]
sr = [15, 16, 17, 12, 13, 15]
        
hw = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
hr = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]

In [ ]:
individual_model = IndividualModel(d, s, sw, sr, hw, hr)
individual_model.solve()
print(individual_model.retailer_cost)
print(individual_model.warehouse_cost)
print(individual_model.total_cost)

In [ ]:
# integrated_model = IntegratedModel(d, s, sw, sr, hw, hr)
# integrated_model.solve()
# integrated_model.total_cost

In [5]:
def ordering_cost(sw=[30, 50], sr=[0.1, 0.3], n_items=5, seed=42):
    sw_min, sw_max = sw
    sr_min, sr_max = sr
    np.random.seed(seed)
    
    sw = np.zeros(n_items)
    sr = np.zeros(n_items)
    
    for i in range(n_items):
        sw[i] = np.random.randint(sw_min, sw_max)
        sr[i] = np.random.randint(sw[i] * sr_min, sw[i] * sr_max)
    
    return sw, sr

def inventory_cost(hw=[0.5, 3.0], hr=[1.2, 2.0], n_items=5, seed=42):
    hw_min, hw_max = hw
    hr_min, hr_max = hr
    np.random.seed(seed)
    
    hw = np.zeros(n_items)
    hr = np.zeros(n_items)
    
    for i in range(n_items):
        hw[i] = (hw_max - hw_min) * np.random.random() + hw_min
        hr[i] = (hr_max*hw[i] - hr_min*hw[i]) * np.random.random() + hr_min*hw[i]
        
    return np.round(hw, 2), np.round(hr, 2)

In [ ]:
times = 12
n_items = 5

random_demands = CreateDemands(500, 5000, size=(times, n_items), seed=42).generate()
random_demands.demands

In [ ]:
def main(s, sw, sr, hw, hr, times, n_items, trials):
    columns = ['S', 'sw', 'sr', 'hw', 'hr', 'M1', 'M2', 'Gap', 'Retail']
    result = pd.DataFrame(index=range(trials), columns=columns)

    sw_range = sw
    sr_range = sr
    hw_range = hw
    hr_range = hr
    
    for i in tqdm.tqdm(range(trials)):
        
        d = CreateDemands(500, 5000, size=(times, n_items), seed=i).generate().demands
        sw, sr = ordering_cost(sw_range, sr_range, n_items=n_items, seed=i)
        hw, hr = inventory_cost(hw_range, hr_range, n_items=n_items, seed=i)
            

        m1 = IndividualModel(d, s, sw, sr, hw, hr)
        m1.solve()
        
        m2 = IntegratedModel(d, s, sw, sr, hw, hr)
        m2.solve()

        result.loc[i, 'S'] = s
        result.loc[i, 'sw'] = sw
        result.loc[i, 'sr'] = sr
        result.loc[i, 'hw'] = hw
        result.loc[i, 'hr'] = hr
        result.loc[i, 'M1'] = m1.total_cost
        result.loc[i, 'M2'] = m2.total_cost
        result.loc[i, 'Gap'] = ((m1.total_cost - m2.total_cost) / m1.total_cost) * 100
        result.loc[i, 'Retail'] = ((m1.total_cost - m2.total_cost) / m1.retailer_cost) * 100

        del m1, m2
        
    return result    

In [ ]:
trials = 3
times = 20
n_items = 15

s = 100
sw = [30, 50]
sr = [0.8, 1.2]
hw = [0.5, 2.0]
hr = [0.8, 1.2]

result = main(s, sw, sr, hw, hr, times, n_items, trials)
result

In [ ]:
import seaborn as sns

temp1 = pd.melt(result, value_vars=['M1','M2'])
temp2 = pd.melt(result, value_vars=['Gap','Retail'])

fig, axes = plt.subplots(1,2, figsize=(6,3), dpi=100)
sns.boxplot(x='variable', y='value', data=temp1, width=0.6, saturation=1.0, ax=axes.flat[0])
sns.boxplot(x='variable', y='value', data=temp2, width=0.6, saturation=1.0, ax=axes.flat[1])
axes.flat[0].grid(axis='y', linestyle='--')
axes.flat[1].grid(axis='y', linestyle='--')
axes.flat[0].set_axisbelow(True)
axes.flat[1].set_axisbelow(True)
axes.flat[0].set_xticklabels(['$M_{1}$', '$M_{2}$'])
axes.flat[0].set_xlabel('Gap between $M_{1}$ and $M_{2}$')
axes.flat[0].set_ylabel('Objective values')
axes.flat[1].set_xlabel('Based on retail')
axes.flat[1].set_ylabel('Percentage ($\%$)')
fig.tight_layout()
plt.show()

In [ ]:
from scipy import stats

trials = 100
times = [10, 15, 20]
n_items = [5, 10, 15]

ss = [100, 200, 300]
sw = [30, 50]
sr = [0.8, 1.2]
hw = [0.5, 2.0]
hr = [0.8, 1.2]

tuples = [(i, t, s) for i in n_items for t in times for s in ss]
index = pd.MultiIndex.from_tuples(tuples)

tuples = [(c1, c2) for c1 in ['Gap', 'Retail'] for c2 in ['min', 'max', 'avg.', 'std.']]
columns = pd.MultiIndex.from_tuples(tuples)
result = pd.DataFrame(index=index, columns=columns)

for i in n_items:
    for t in times:
        for s in ss:
            tmp = main(s, sw, sr, hw, hr, t, i, trials)
            result.loc[(i, t, s), ('Gap', 'min')] = '{:.2f}%'.format(tmp['Gap'].min())
            result.loc[(i, t, s), ('Gap', 'max')] = '{:.2f}%'.format(tmp['Gap'].max())
            result.loc[(i, t, s), ('Gap', 'avg.')] = '{:.2f}%'.format(tmp['Gap'].mean())
            result.loc[(i, t, s), ('Gap', 'std.')] = '{:.2f}%'.format(tmp['Gap'].std())
            result.loc[(i, t, s), ('Gap', 'ttest')] = stats.ttest_rel(tmp['M1'].values, tmp['M2'].values)[0]
            result.loc[(i, t, s), ('Gap', 'p-value')] = stats.ttest_rel(tmp['M1'].values, tmp['M2'].values)[1]
            result.loc[(i, t, s), ('Retail', 'min')] = '{:.2f}%'.format(tmp['Retail'].min())
            result.loc[(i, t, s), ('Retail', 'max')] = '{:.2f}%'.format(tmp['Retail'].max())
            result.loc[(i, t, s), ('Retail', 'avg.')] = '{:.2f}%'.format(tmp['Retail'].mean())
            result.loc[(i, t, s), ('Retail', 'std.')] = '{:.2f}%'.format(tmp['Retail'].std())

In [ ]:
# result.to_excel('result-01.xlsx')
result

In [6]:
class IntegratedModel:
    def __init__(self, d, s, sw, sr, hw, hr):
        self.d = d
        self.n_items = d.shape[0]
        self.times = d.shape[1]
        
        self.s = s
        self.sw = sw
        self.sr = sr
        
        self.hw = hw
        self.hr = hr
        
        self.m = d.sum().sum()
        print(self.m)
        
        self.verbose = False
        
    def create_problem(self):
        idx = [(i, t) for i in range(self.n_items) for t in range(self.times)]
        
        # Define model
        prob = LpProblem('Warehouse', LpMinimize)
        
        # Decision variables
        qw = LpVariable.dicts('Qw', idx, lowBound=0, cat='Continuous') # 240
        qr = LpVariable.dicts('Qr', idx, lowBound=0, cat='Continuous') # 240
        
        y = LpVariable.dicts('Y', list(range(self.times)), cat='Binary') # 12
        zw = LpVariable.dicts('Zw', idx, cat='Binary') # 240
        zr = LpVariable.dicts('Zr', idx, cat='Binary') # 240
        
        iw = LpVariable.dicts('Iw', idx, lowBound=0, cat='Continuous') # 240
        ir = LpVariable.dicts('Ir', idx, lowBound=0, cat='Continuous') # 240

        # Objective function
        prob += lpSum(y[t] for t in range(self.times)) * self.s \
                + lpSum([self.sw[i] * zw[i, t] for i in range(self.n_items) for t in range(self.times)]) \
                + lpSum([self.sr[i] * zr[i, t] for i in range(self.n_items) for t in range(self.times)]) \
                + lpSum([self.hw[i] * iw[i, t] for i in range(self.n_items) for t in range(self.times)]) \
                + lpSum([self.hr[i] * ir[i, t] for i in range(self.n_items) for t in range(self.times)])
        
        # 기초재고식
        for i in range(self.n_items):
            prob += 0 + qw[i, 0] - qr[i, 0] == iw[i, 0]
            prob += 0 + qr[i, 0] - self.d.iloc[i, 0] == ir[i, 0]

        # 기말재고식
        for i in range(self.n_items):
            for t in range(1, self.times):
                prob += iw[i, t-1] + qw[i, t] - qr[i, t] == iw[i, t]
                prob += ir[i, t-1] + qr[i, t] - self.d.iloc[i, t] == ir[i, t]

        for t in range(self.times):
            prob += lpSum(qw[i, t] for i in range(self.n_items)) <= y[t] * self.m

        for i in range(self.n_items):
            for t in range(self.times):
                prob += lpSum(qw[i, t]) <= zw[i, t] * self.m
                prob += lpSum(qr[i, t]) <= zr[i, t] * self.m
        
        return prob
            
    def solve(self):
        prob = self.create_problem()
        variables = []
        constraints = []
        for v in prob.variables():
            variables.append(v.name)
            
        for c in prob.constraints:
            constraints.append(c)
        
        print(len(variables), len(constraints))
        print()
        prob.solve(solver=GUROBI(msg=False, epgap=0.0))
        if self.verbose:
            print('Integrated model: {}'.format(LpStatus[prob.status]))
        
        self.total_cost = value(prob.objective)
    


In [9]:
def main(s, sw, sr, hw, hr, times, n_items, trials):
    columns = ['S', 'sw', 'sr', 'hw', 'hr', 'M1', 'M2', 'Gap', 'Retail']
    result = pd.DataFrame(index=range(trials), columns=columns)

    sw_range = sw
    sr_range = sr
    hw_range = hw
    hr_range = hr
    
    for i in range(trials):
        
        d = CreateDemands(500, 5000, size=(times, n_items), seed=i).generate().demands
        sw, sr = ordering_cost(sw_range, sr_range, n_items=n_items, seed=i)
        hw, hr = inventory_cost(hw_range, hr_range, n_items=n_items, seed=i)
        print(d.sum().sum())
        print(sw.sum(), sr.sum())
        print(hw.sum(), hr.sum())

            

        m1 = IndividualModel(d, s, sw, sr, hw, hr)
        m1.solve()
        
        m2 = IntegratedModel(d, s, sw, sr, hw, hr)
        m2.solve()

        result.loc[i, 'S'] = s
        result.loc[i, 'sw'] = sw
        result.loc[i, 'sr'] = sr
        result.loc[i, 'hw'] = hw
        result.loc[i, 'hr'] = hr
        result.loc[i, 'M1'] = m1.total_cost
        result.loc[i, 'M2'] = m2.total_cost
        result.loc[i, 'Gap'] = ((m1.total_cost - m2.total_cost) / m1.total_cost) * 100
        result.loc[i, 'Retail'] = ((m1.total_cost - m2.total_cost) / m1.retailer_cost) * 100
        
    return m1, m2, result

In [12]:
trials = 10
times = 20
n_items = 15

s = 100
sw = [30, 50]
sr = [0.8, 1.2]
hw = [0.5, 2.0]
hr = [0.8, 1.2]


model1, model2, result = main(s, sw, sr, hw, hr, times, n_items, trials)
#     result.loc[(15, 20, s), ('Gap', 'min')] = '{:.2f}%'.format(tmp['Gap'].min())
#     result.loc[(15, 20, s), ('Gap', 'max')] = '{:.2f}%'.format(tmp['Gap'].max())
#     result.loc[(15, 20, s), ('Gap', 'avg.')] = '{:.2f}%'.format(tmp['Gap'].mean())
#     result.loc[(15, 20, s), ('Gap', 'std.')] = '{:.2f}%'.format(tmp['Gap'].std())
#     result.loc[(15, 20, s), ('Gap', 'ttest')] = stats.ttest_rel(tmp['M1'].values, tmp['M2'].values)[0]
#     result.loc[(15, 20, s), ('Gap', 'p-value')] = stats.ttest_rel(tmp['M1'].values, tmp['M2'].values)[1]
#     result.loc[(15, 20, s), ('Retail', 'min')] = '{:.2f}%'.format(tmp['Retail'].min())
#     result.loc[(15, 20, s), ('Retail', 'max')] = '{:.2f}%'.format(tmp['Retail'].max())
#     result.loc[(15, 20, s), ('Retail', 'avg.')] = '{:.2f}%'.format(tmp['Retail'].mean())
#     result.loc[(15, 20, s), ('Retail', 'std.')] = '{:.2f}%'.format(tmp['Retail'].std())

42175
605.0 582.0
18.66 19.9
42175
1820 1220

43711
613.0 585.0
14.410000000000002 15.149999999999997
43711
1820 1220

46289
554.0 552.0
16.58 15.479999999999999
46289
1820 1220

43274
582.0 549.0
15.780000000000003 16.12
43274
1820 1220

37359
561.0 544.0
20.180000000000003 20.15
37359
1820 1220

46367
594.0 594.0
17.64 17.1
46367
1820 1220

43157
608.0 585.0
19.6 20.060000000000002
43157
1820 1220

48033
607.0 582.0
19.03 18.88
48033
1820 1220

41434
594.0 589.0
17.229999999999997 17.93
41434
1820 1220

40720
592.0 567.0
19.62 19.460000000000004
40720
1820 1220



GurobiError: Model too large or QP not enabled for current Gurobi license

In [ ]:
trials = 100
times = 20
n_items = 15

s = 100
sw = [30, 50]
sr = [0.8, 1.2]
hw = [0.5, 2.0]
hr = [0.8, 1.2]

for s in [100, 200, 300]:
    tmp = main(s, sw, sr, hw, hr, times, n_items, trials)
    result.loc[(15, 20, s), ('Gap', 'min')] = '{:.2f}%'.format(tmp['Gap'].min())
    result.loc[(15, 20, s), ('Gap', 'max')] = '{:.2f}%'.format(tmp['Gap'].max())
    result.loc[(15, 20, s), ('Gap', 'avg.')] = '{:.2f}%'.format(tmp['Gap'].mean())
    result.loc[(15, 20, s), ('Gap', 'std.')] = '{:.2f}%'.format(tmp['Gap'].std())
    result.loc[(15, 20, s), ('Gap', 'ttest')] = stats.ttest_rel(tmp['M1'].values, tmp['M2'].values)[0]
    result.loc[(15, 20, s), ('Gap', 'p-value')] = stats.ttest_rel(tmp['M1'].values, tmp['M2'].values)[1]
    result.loc[(15, 20, s), ('Retail', 'min')] = '{:.2f}%'.format(tmp['Retail'].min())
    result.loc[(15, 20, s), ('Retail', 'max')] = '{:.2f}%'.format(tmp['Retail'].max())
    result.loc[(15, 20, s), ('Retail', 'avg.')] = '{:.2f}%'.format(tmp['Retail'].mean())
    result.loc[(15, 20, s), ('Retail', 'std.')] = '{:.2f}%'.format(tmp['Retail'].std())
    del tmp

In [ ]:
result

In [ ]:
result.loc[(15, 20, 100), ('Gap', 'min')]

In [ ]:
for i in [100, 200, 300]:
    